In [2]:
import chess.pgn
import io
import json

In [45]:
# 1. Add fens to files
file_no_fens = './openings-no-variations.json'
output_file = './openings_novar.json'

with open(file_no_fens) as f:
    openings = json.load(f)
    for opening in openings:
        pgn = opening['pgn']
        game = chess.pgn.read_game(io.StringIO(pgn))
        board = game.board()
        for move in game.mainline_moves():
            board.push(move)
        opening['fen'] = board.fen()

    # dump to file
    with open(output_file, 'w') as f:
        json.dump(openings, f, indent=4, ensure_ascii=True)

In [ ]:
# TODO: 2. using file contains FENs, call lichess api, appending winrates
# see ./openings_novar.json 

In [3]:
# 3. 
# - Filter out openings with identical names, taking the smallest PGN string (more generic opening PGN)
# - Sort the list by popularity: total number of games from largest to smallest

# dict, mapping opening name => opening object
# with each insertion, check 

opening_dict = {} 

with open('./openings_novar.json', 'r') as f:
    openings = json.load(f)
    for opening in openings:
        name = opening['name']
        exists = name in opening_dict 
        # add to dictionary if:
        # not in dictionary already, or
        # in dictionary already, but the new opening has a smaller pgn
        if(not exists or (exists and opening['pgn'] < opening_dict[name]['pgn'])):
            # add number of games played to object (sum of winrates)
            opening['games'] = sum(opening['winrate'])
            # add to dict
            opening_dict[name] = opening
            
    # sort by games played
    no_duplicates = list(opening_dict.values())
    no_duplicates.sort(key=lambda o: -o['games'])

# write to file
with open('./openings_novar_sorted.json', 'w') as f:
    f.write(json.dumps(no_duplicates, indent=4))